# Lesson 16 - EM and Gaussian Mixture Models


## Objectives
- Implement EM for a Gaussian mixture model.
- Track log-likelihood across iterations.
- Visualize soft cluster assignments.


## From the notes

**EM for GMM**
- E-step: compute responsibilities $w_i^{(j)}$.
- M-step: update $\phi_j, \mu_j, \Sigma_j$ using responsibilities.

_TODO: Validate EM updates in the CS229 main notes PDF._


## Intuition
EM alternates between estimating cluster responsibilities and updating parameters to maximize expected log-likelihood.


## Data
We generate a mixture of two Gaussians in 2D.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

X1 = np.random.multivariate_normal([-2, -2], np.eye(2), 120)
X2 = np.random.multivariate_normal([2, 2], np.eye(2), 120)
X = np.vstack([X1, X2])

def gaussian_pdf(X, mean, cov):
    d = X.shape[1]
    diff = X - mean
    inv = np.linalg.pinv(cov)
    exp = np.exp(-0.5 * np.sum(diff @ inv * diff, axis=1))
    return exp / np.sqrt((2 * np.pi) ** d * np.linalg.det(cov))

def em_gmm(X, k=2, iters=20):
    n, d = X.shape
    idx = np.random.choice(n, k, replace=False)
    mu = X[idx]
    cov = np.array([np.eye(d) for _ in range(k)])
    phi = np.ones(k) / k
    log_lik = []
    for _ in range(iters):
        # E-step
        resp = np.stack([phi[j] * gaussian_pdf(X, mu[j], cov[j]) for j in range(k)], axis=1)
        resp = resp / resp.sum(axis=1, keepdims=True)
        # M-step
        Nk = resp.sum(axis=0)
        phi = Nk / n
        mu = (resp.T @ X) / Nk[:, None]
        cov = np.array([
            (resp[:, j][:, None] * (X - mu[j])).T @ (X - mu[j]) / Nk[j]
            for j in range(k)
        ])
        log_lik.append(np.sum(np.log(resp.sum(axis=1))))
    return resp, mu, cov, log_lik

resp, mu, cov, log_lik = em_gmm(X)


## Experiments


In [ ]:
log_lik[-1]


## Visualizations


In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(X[:,0], X[:,1], c=resp[:,0], cmap="coolwarm", alpha=0.7)
plt.scatter(mu[:,0], mu[:,1], c="black", marker="x", s=100)
plt.title("GMM responsibilities")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()

plt.figure(figsize=(6,4))
plt.plot(log_lik)
plt.title("EM log-likelihood")
plt.xlabel("iteration")
plt.ylabel("log-likelihood")
plt.show()


## Takeaways
- EM provides a monotonic increase in log-likelihood for mixture models.
- Responsibilities reveal soft cluster assignments.


## Explain it in an interview
- Explain the E-step and M-step in EM.
- Describe how k-means relates to GMM EM.


## Exercises
- Increase k and visualize the responsibilities.
- Initialize EM with k-means centroids.
- Compare EM results with different seeds.
